# Auto-segmentation workshop (April 2023)

This notebook will show, briefly, how to build an autosegmentation model for abdominal organs using pytorch and pytorch-lightning. We will be using some open data from the [MICCAI 2021 FLARE Challenge](https://flare.grand-challenge.org/). This dataset contains ~500 patients, each of which has four OARs segmented.

Typically, you would need to read DICOM images (using a library like `pydicom`) but, for simplicity, we have already done this and have converted all images into numpy arrays.  

We will be using a suite of pre-built pytorch segmentation models in the excellent [segmentation-models](https://github.com/qubvel/segmentation_models.pytorch) package. This package simplifies the building of pretrained segmentation models in 2D. Again, for simplicity, we will be doing segmentation in 2D instead of 3D - although everything you learn today is transferable to 3D.

Pytorch can be quite intimidating, but is very powerful when you get to grips with it. In the interests of simplicity, we will use a wrapper around pytorch called [pytorch-lightning](https://pytorch-lightning.readthedocs.io/en/latest/). Lightning separates out the intricacies, allowing you to write a bit less boilerplate code, and letting us very quickly and easily use best-practice methods to train our models.

---

# Overview
The steps in this notebook are as follows:

0. Install prerequisites and set up;
1. Load data;
2. Define some preprocessing and apply it to the CT slices;
3. Create a segmentation model using a library containing pre-trained models;
4. Train the model on the training examples;
5. Test the model against the testing data, evaluate model accuracy and visualise the results.

---

# How to use Colab 
Colab is a free ML playground from google. It allows you free access to limited resources, including a GPU and some storage space. Even though the limits are quite small: ~30GB disk, 12GB RAM & a random GPU from K80 up to P100, you can do some pretty cool stuff with it. __You will need a Google account to sign into it.__

*Important*:
We are going to be training a Convolutional Neural Network (CNN), so we need to get a GPU. To do this, click "Runtime" in the menu across the top of the colab page, then select "Change Runtime Type". From the dropdown, select GPU and click Save. The runtime will then reboot and you should have a GPU. To find out what you got, run the cell below this text.

In [ ]:
## Find out what GPU we got (and make sure we actually have one!)
!nvidia-smi

Colab notebooks are identical to Jupyter notebooks. For help click [here](https://colab.research.google.com/github/rrr-uom-projects/autoseg_workshop_2023/blob/main/Jupyter_help.ipynb)

---

# Loading data

The cell below downloads all the necessary data and places it on the colab machine.

In [ ]:
## Download all the data! This may take a little while...

## Processed data
try:
    import google.colab
    IN_COLAB = True
    !wget https://www.dropbox.com/s/la05h49y9ths7x3/autoseg_data.tar.gz?dl=0 -O /content/AllProcessedData.tar
    !tar -xf /content/AllProcessedData.tar -C /content/
    !rm /content/AllProcessedData.tar
except:
    IN_COLAB = False
    !wget https://www.dropbox.com/s/la05h49y9ths7x3/autoseg_data.tar.gz?dl=0 -O ./AllProcessedData.tar
    !tar -xf ./AllProcessedData.tar -C ./
    !rm AllProcessedData.tar

# utilities python file
if IN_COLAB:
    !wget https://www.dropbox.com/s/rng7h9mgkwaolt8/utils.py?dl=0 -O /content/utils.py
else:
    !wget https://www.dropbox.com/s/rng7h9mgkwaolt8/utils.py?dl=0 -O ./utils.py

## 0. Install prerequisites

Here we install the segmentation-models library, along with pytorch-lightning and tqdm (for fancy progress bars).

In [ ]:
if IN_COLAB:
    %pip install git+https://github.com/qubvel/segmentation_models.pytorch
    %pip install pytorch_lightning tqdm

## 0. Set up monitoring and enable matplotlib notebook interactivity

In [ ]:
%matplotlib inline
import tensorboard
%load_ext tensorboard

## 0. Load required libraries

In [ ]:
import segmentation_models_pytorch as smp
import matplotlib.pyplot as plt 
import pytorch_lightning as pl
import albumentations as A
import numpy as np
# import pydicom
import torch
import os
from utils import getFiles
from os.path import join
import pickle
## These are the structures defined in the FLARE data
structure_names = ['Body', 'Liver', 'Kidneys', 'Spleen', 'Pancreas']
## See if we're in colab...
if IN_COLAB:
    datapath = "/content/FLARE_data/"
else:
    datapath = "/home/ed/autoseg_workshop_2023/FLARE_data/"  ## <---- You will need to change this if running locally


# 1. Load and visualise the data
- The most basic CNNs classify an image into one of several classes (e.g. dog, bird, cat etc). This classification is across the whole image, and if there were to be an image containing both a dog and cat, it would be difficult to classify that image, since the network can "see" both things. 
- Object detection networks are slightly more complex and work by drawing a bounding box around each object in the image - this means they can handle images with more than one class in.
- In radiotherapy, or lots of other tasks, we need to know exactly which regions of the image are what object - this is where segmentation, or semantic segmentation comes in. Semantic segmentation gives a class label to every pixel in the image, allowing it to accurately track the edges of organs and other things.

Earlier, we downloaded and decompressed data for us to use (you should see these in the sidebar to the left). This includes both the images and the masks labelling each structure. For the masks, the pixel value is based on the index of the structure. Usually you would need to derive this from the DICOM RTSTRUCT file, but we have bypassed this step for simplicity. In other words, our masks are numpy arrays with the same dimensionality as the input image but instead of intensity values, each pixel is an integer value representing which structure this pixel "belongs" to.

The following cell, loads the pre-processed data:

In [ ]:
## Get the training data
train_datapath = join(datapath, "train")
fnames = sorted(getFiles(join(train_datapath, 'ims')))
train_ct_slices = np.zeros((len(fnames), 256, 256), dtype=np.float32)
train_mask_slices = np.zeros((len(fnames), 256, 256), dtype=np.float32)


for fdx, fname in enumerate(fnames):
    train_ct_slices[fdx] = np.load(join(train_datapath, "ims", fname))
    train_mask_slices[fdx] = np.load(join(train_datapath, "masks", fname))

# load pixel spacing
with open(join(train_datapath, "spacings.pkl"), "rb") as f:
    train_pixel_sizes = pickle.load(f)


# Get the test data
test_datapath = join(datapath, "test")
fnames = sorted(getFiles(join(test_datapath, 'ims')))
test_ct_slices = np.zeros((len(fnames), 256, 256), dtype=np.float32)
test_mask_slices = np.zeros((len(fnames), 256, 256), dtype=np.float32)

for fdx, fname in enumerate(fnames):
    test_ct_slices[fdx] = np.load(join(test_datapath, "ims", fname))
    test_mask_slices[fdx] = np.load(join(test_datapath, "masks", fname))

# load pixel spacing
with open(join(test_datapath, "spacings.pkl"), "rb") as f:
    test_pixel_sizes = pickle.load(f)

print(train_ct_slices.shape)
print(train_mask_slices.shape)
print(test_ct_slices.shape)
print(test_mask_slices.shape)

# 2. Preprocessing

Preprocessing your data is an extremely important part of machine learning. Preprocessing is used to standardise the images and, especially in machine learning, to compress their intensities down to a given range, which helps model training.

The preprocessing performed in the next cell is the simplest and most basic we could think of. We first standardise the images by applying a level/window transformation and map the resulting pixel values to floating point values in the range 0-1.

In [ ]:
def window_level(data, window=350, level=50):
    """
    Apply a window and level transformation to CT slices. 

    The default values are taken taken from https://radiopaedia.org/articles/windowing-ct?lang=gb
    
    Pixel values will be in the range 0-1
    """
    ## calculate high & low edges of level & window
    low_edge  = level - (window//2)
    high_edge = level + (window//2)
    ## use np.clip to clip into that level/window, then adjust to range 0 - 1
    windowed_data = (np.clip(data, low_edge, high_edge) - low_edge)/window

    return windowed_data

Now we can apply this transformation to the data we loaded. This shouldn't take too long...

In [ ]:
## Apply the preprocessing 
window_levelled_slices_train = window_level(train_ct_slices)
window_levelled_slices_test = window_level(test_ct_slices)

## If you run out of RAM, you can save memory by deleting the original array - will need to re-load it if we change something
# del train_annotated_ct_slices
# del test_annotated_ct_slices

print(window_levelled_slices_train.shape)
print(window_levelled_slices_test.shape)

## Sanity checking
It is a **very good** idea to periodically check that your data actually looks as you expect it. Lots of issues can be solved by double-checking your inputs! In imaging, this can be as simple as plotting the CT slice with the masks overlaid and making sure they line up. Let's do this quickly now:

In [ ]:
# A simple sanity check plot  

n=47
fig, ax = plt.subplots()
ax.imshow(window_levelled_slices_train[n,...], cmap='Greys_r')
ax.imshow(train_mask_slices[n,...].squeeze(), alpha=0.5, cmap='viridis', vmax=5)
ax.invert_yaxis()
plt.show()

If you've been paying attention, you will know that we have ~5000 images to train on. That's a lot. To make training a bit more tractable, I will now randomly select ~1500 training examples and ~500 validation ones. Ideally, you would just use the whole dataset, but we will either run out of memory or time if we do. We set the numpy random seed to a known value so everyone should get roughly the same results, then re-seed randomly afterwards so nothing else is affected.

In [ ]:
np.random.seed(1234)
# Randomly select 1500 indices used to extract a subset of images
subset_indices = np.random.randint(0, window_levelled_slices_train.shape[0], size=1500)

# Extract training subset
wl_slice_subset_train = window_levelled_slices_train[subset_indices[0:1000]]
mask_subset_train = train_mask_slices[subset_indices[0:1000]]
spacings_subset_train = np.array(list(train_pixel_sizes.values()))[subset_indices[0:1000]]

# Validation subset
wl_slice_subset_val = window_levelled_slices_train[subset_indices[-500:]]
mask_subset_val =  train_mask_slices[subset_indices[-500:]]
spacings_subset_val = np.array(list(train_pixel_sizes.values()))[subset_indices[-500:]]

# Uncomment the lines below if you need to clear up memory
# del window_levelled_slices_train
# del train_mask_slices

np.random.seed()

In [ ]:
## Another sanity check to be sure that did what we expected...

n=7
fig, ax = plt.subplots()
ax.imshow(wl_slice_subset_train[n,...], cmap='Greys_r')
ax.imshow(mask_subset_train[n,...].squeeze(), alpha=0.5, cmap='viridis', vmax=5)
ax.invert_yaxis()
print(wl_slice_subset_train.shape)
plt.show()

## Dataset creation 

In the cell below, we create a subclass of the pytorch `Dataset` object. This is used to create an iterable object that can be used during training to randomly sample batches of inputs and labels and randomly apply transformations (data augmentation). The `Dataset` class is built with three arguments - the array of images, the array of masks and a set of transformations we wish to apply.

These transformations are where we can introduce some data augmentation. Data augmentation is the process of applying random transformations to our data to create "new" synthetic data to train our model with. This allows us to sample the underlying distribution generating our data, without needing to collect more images. This improves model training by reducing overfitting and improving generalisability. 

There are also some transformations that are mandatory. For example, pytorch's pretrained models expect images to be of type float, and have a specific mean and standard deviation (because they were pre-trained on the imagenet dataset). Therefore, We apply a transformation that handles normalising to the imagenet mean and standard deviation (`Normalize`). 

We will use a library called `albumentations` to handle our augmentations. It is very fast and easy to use, but only works for 2D images. For 3D augmentations, there is a library called `kornia` which can do some augmentations.

In [ ]:
## Define a subclass of Dataset that handles our image-mask pair loaded from arrays
## This is the bare minimum example!
class LCTSCDataGen(torch.utils.data.Dataset):
  def __init__(self, image_array, mask_array, transform, spacings):
    super().__init__()
    self.image_array = image_array
    self.mask_array = mask_array
    self.transform = transform
    self.spacings = spacings

  def __len__(self):
    return self.image_array.shape[0]

  def __getitem__(self, idx):
    image = self.image_array[idx,...]
    mask = self.mask_array[idx, ...]
    if self.transform is not None:
      transformed = self.transform(image=image, mask=mask)
      image = transformed['image']
      mask = transformed['mask']
    spacing = self.spacings[idx]
    return image[np.newaxis,...], mask, spacing

## Now create the augmentation pipeline
train_transforms = A.Compose([
    A.Rotate(5),
    #<<<<<< PLAY AROUND WITH MORE AUGMENTATIONS IF YOU WANT >>>>>>>>>>>>>>>>>>>
    A.Normalize(mean=(np.mean([0.485, 0.456, 0.406])), std=(np.mean([0.229, 0.224, 0.225]))) ## Note mean of means, mean of stds
])

## valdation pipeline just does normalisation and conversion to tensor
val_transforms = A.Compose([
    # <<<<<< NOTICE NO AUGMENTATIONS HERE! >>>>>>>>>
    A.Normalize(mean=(np.mean([0.485, 0.456, 0.406])), std=(np.mean([0.229, 0.224, 0.225]))) ## Note mean of means, mean of stds
])

**Important**:
We use different pipelines for the training and validation data. For training, we can do whatever we like to make the most of our data, but when validating, we're meant to be getting an idea of the network's performance on 'real' data, so we should not do any augmentation.

In [ ]:
## Now create some datasets and dataloaders
train_dataset = LCTSCDataGen(wl_slice_subset_train, mask_subset_train, train_transforms, spacings_subset_train)
val_dataset = LCTSCDataGen(wl_slice_subset_val, mask_subset_val, val_transforms, spacings_subset_val)

## Create dataloaders from these datasets
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=8)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=8)

print(mask_subset_train.shape, wl_slice_subset_train.shape)

In [ ]:
## One more sanity check... Again this is important to do!
train_img, train_msk, _ = next(iter(train_dataloader))

fig, ax = plt.subplots()
# Notice we need to convert our data back to numpy arrays. 
# This is because Pytorch uses tensors to calculate gradients, backpropagate errors and optimise the model.
ax.imshow(train_img.numpy()[3,...].squeeze(), cmap='Greys_r')
ax.imshow(train_msk.numpy()[3,...].squeeze(), alpha=0.5, cmap='viridis', vmax=5)
ax.invert_yaxis()
plt.show()


---

# 3. Setting up model training
### Segmentation model

For simplicity, we are going to use a python library of pre-built segmentation networks. The library is called `segmentation_models` and I highly recommend you have a look at the [github page](https://github.com/qubvel/segmentation_models.pytorch). We already imported the library at the top of the notebook, so we can use it straight away here.

We will be using a pre-trained segmentation model, in which the feature extraction parts of the network have been pre-trained on imagenet (A large database of natural images). This means we can train a good model with relatively little data, but also means we have to convert our 1 channel CT image into a 3 channel RGB image. Fortunately, the library can do the 1 -> 3 channel conversion for us by repeating the image in the channel dimension.

We will be using an architecture called a Feature Pyramid Network or [FPN](), it looks something like this:

![FPN architecture, from segmentation_models github page](https://github.com/qubvel/segmentation_models/raw/master/images/fpn.png)

The part on the left, in gray, is the bit that is pretrained on imagenet; this is called the backbone of the network. Pretraining means that the backbone was previously trained on another task where more data is available (in this case natural image classification) and has already learned some useful features on that task. We will use these features as a starting point from which we will learn features useful to our task. This massively decreases the amount of data we need - which is vital in medical imaging.

The choice of backbone is somewhat arbitrary - more modern and bigger networks should have better performance, but not always. We will use the smallest network available, ResNet-18 because it will train faster and use less GPU memory. 

### Training loop

Writing your own training loop is relatively straightforward in Pytorch, but for simplicity we are going to use a library called `pytorch-lightning` to abstract this away (In Part 3, you can try to write your own loops in Pytorch). `Pytorch-lightning` works by wrapping our model in a special class that inherits from a LightningModule. To do the wrapping, we create a class LightningFPN, and define a few mandatory functions for pytorch-lightning to use. *Don't worry if this doesn't make sense - try to follow the code!*

#### Optimiser
While creating the wrapper, we also have to select which optimiser to use. There are lots of considerations in selecting an optimiser, but most people use one of either Stochastic Gradient Descent (SGD) with momentum, or Adaptive Moment Estimation (Adam). Usually, Adam converges faster than SGD, but sometimes SGD can find a better solution by avoiding a local minimum. In the interests of speed, we will use Adam. *This is something you can play around with if you want!*

#### Loss function
The choice of loss function has a profound impact on the quality of the trained model.

Segmentation is just a classification problem applied to every pixel in the image, so we could use a loss designed for classification, apply it in every pixel and then take the average across the whole image. This is what the categorical crossentropy loss will do. Categorical crossentropy can fall over though - especially when there is a class imbalance. Since we are segmenting organs that occupy a small fraction of the image, we have a lot of background and not much foreground, therefore the loss will be dominated by the background (i.e. this will drive the model to predict everything as background!). There are ways around this (e.g. weighted cross entropy, or focal loss), but we'll use something else.

The Dice similarity coefficient (DSC) is a very well known and widely used measure of overlap. **DSC is rightly villified in radiotherapy because it has no spatial component (how far apart were the contours) and is very sensitive to volume (small contours will always be much worse than big ones).** For simplicity's sake, we will ignore these issues and use DSC. The segmentation_models library has implementations of all these losses so it would be trivial to use one of the other losses. *Again this is something you can play around with!*


In [ ]:
## Define the class that will wrap te pytorch model up for ptl
class LightningFPN(pl.LightningModule):
  def __init__(self):
    super().__init__()
    ## Create the pytorch model 
    self.model = smp.FPN("resnet18", in_channels=1, classes=len(structure_names)+1, encoder_weights='imagenet')
    
    ## Construct a loss function, this is DSC, configured for multiple classes, and ignoring the background
    self.loss_fcn = smp.losses.DiceLoss("multiclass", from_logits=True)

    ## Specify which optimiser to use here
    self.optimizer = torch.optim.Adam

  def forward(self, x):
    return self.model(x)

  def configure_optimizers(self):
    optimizer = self.optimizer(self.parameters(), lr=1e-4)## May need to handle other kwargs here!
    return {"optimizer": optimizer, "reduce_on_plateau":True}
    ## Note - we are reducing the learning rate when the validation loss plateaus for a while - this should improve the model

  def training_step(self, batch, batch_idx):
    # Separate batch into input, mask and spacing (although we ignore the spacing)
    img, msk, _ = batch
    # Pass the input through the model and get a prediction (msk_hat)
    msk_hat = self(img)
    # Calculate average prediction error on this batch
    loss = self.loss_fcn(msk_hat, msk.long())
    # Log the error 
    self.log("loss", loss)
    return loss

  def validation_step(self, batch, batch_idx):
    # Identical to the training step
    img, msk, _ = batch
    msk_hat = self(img)
    val_loss = self.loss_fcn(msk_hat, msk.long())
    self.log("val_loss", val_loss)
    return val_loss
  
## Now we can wrap the prebuilt model up inside a pytorch lightning module:
pl_model = LightningFPN()

## Done!

This model has about 13 million parameters, which might stretch our GPU a bit - we will need to think about batch size if this becomes an issue during training.


---

# 4. Training

We're now ready to train the model. It is very easy when using pytorch-lightning, it only takes 2 lines to do what would take dozens in pure pytorch!

To be able to keep track of what is going on, we will use the tensorboard log viewer, which I activate in the next cell. This will allow us to see the training and validation loss change as the network learns.

To keep things quick, we will only train for 5 epochs. Ideally, we would train for a few hundred, to make sure the network loss is properly saturated.

In [ ]:
%tensorboard --logdir lightning_logs/

In [ ]:
trainer = pl.Trainer(max_epochs=5)
trainer.fit(pl_model, train_dataloader, val_dataloader)

**The cell below can be used to save/reload models if Colab crashes**

In [ ]:
# save a checkpoint in case colab crashes
trainer.save_checkpoint("abdominal_model.ckpt")

# code to reload the model if colab crashes
#pl_model = LightningFPN.load_from_checkpoint("model.ckpt")

### Interim check:

Check model predictions on a random example.

In [ ]:
in_image, in_mask, in_spacing = next(iter(val_dataloader))

# take the first example from the batch
in_image = in_image[0].unsqueeze(0)
all_gt = in_mask[0].numpy().astype(float)
in_spacing = in_spacing[0].numpy()

all_gt[all_gt == 0.0] = np.nan ## make the background invisible
test = pl_model.model.predict(in_image).detach() 
# The model doesn't output the class each pixel belongs to but rather an un-normalized weight for each class.  
# To convert back to the class indices we use the softmax activation. 
probs = torch.nn.functional.softmax(test, dim=1)[0].numpy().squeeze()
test_mask = np.argmax(probs, axis=0).astype(float)
test_mask[test_mask == 0.0] = np.nan


## Show the results
fig =  plt.figure(figsize=(10,10)) 
ax_gt = fig.add_subplot(121)
ax_gt.set_title("Ground Truth")
ax_cnn = fig.add_subplot(122)
ax_cnn.set_title("CNN contour")

ax_gt.imshow(in_image.squeeze(), cmap='Greys_r')
ax_gt.imshow(all_gt.squeeze(), alpha=0.75, cmap='viridis', vmin=0, vmax=5)
ax_gt.invert_yaxis()

ax_cnn.imshow(in_image.squeeze(), cmap='Greys_r')
ax_cnn.imshow(test_mask, alpha=0.75, cmap='viridis', vmin=0, vmax=5)
ax_cnn.invert_yaxis()

If training a segmentation model doesn't work in colab, I have pre-trained a model on the exact same data as here, but for 200 epochs. You can uncomment the cell below to load it and check a random prediction.

In [ ]:
# model_trained = pl_model.load_from_checkpoint(os.path.join(datapath, "pretrained_checkpoint.ckpt"))

# in_image, in_mask, in_spacing = next(iter(val_dataloader))

# # take the first example from the batch
# in_image = in_image[0].unsqueeze(0)
# all_gt = in_mask[0].numpy().astype(float)
# in_spacing = in_spacing[0].numpy()

# all_gt[all_gt == 0.0] = np.nan ## make the background invisible
# test = model_trained.model.predict(in_image).detach()
# probs = torch.nn.functional.softmax(test, dim=1)[0].numpy().squeeze()
# test_mask = np.argmax(probs, axis=0).astype(float)
# test_mask[test_mask == 0.0] = np.nan


# ## Show the results
# fig =  plt.figure(figsize=(10,10)) 
# ax_gt = fig.add_subplot(121)
# ax_gt.set_title("Ground Truth")
# ax_cnn = fig.add_subplot(122)
# ax_cnn.set_title("CNN contour")

# ax_gt.imshow(in_image.squeeze(), cmap='Greys_r')
# ax_gt.imshow(all_gt.squeeze(), alpha=0.75, cmap='viridis', vmin=1, vmax=5)
# ax_gt.invert_yaxis()

# ax_cnn.imshow(in_image.squeeze(), cmap='Greys_r')
# ax_cnn.imshow(test_mask, alpha=0.75, cmap='viridis', vmin=1, vmax=5)
# ax_cnn.invert_yaxis()

# 5. Evaluation

Visually, the results look pretty good, but now let's try to quantify it a bit. We're going to use a few metrics: 
- Dice Coefficient - the same as our loss. Measures overlap and is sensitive to structure volume. **In practice, you should never evaluate your model on the metric you have optimised as this will bias your results.**
- Mean surface distance - The mean of the directed Hausdorff distance (HD) from GT -> CNN and CNN -> GT
- 95th percentile Hausdorff distance - the mean of the directed 95th percentile HD from GT -> CNN and CNN -> GT


The cell below iterates through all the structures in the test slice and calculates each metric against the corresponding ground-truth.

In [ ]:
from utils import compute_dice_coefficient, compute_surface_distances, compute_average_surface_distance, compute_robust_hausdorff

distance_measures = {name : {} for name in structure_names[1:]}
organ_labels = [2,3,4,5]

for name, organ_label in zip(structure_names[1:], organ_labels):

    cnn_prediction = test_mask == organ_label
    gt = all_gt == organ_label

    if np.sum(gt) == 0:
        print(f"Skipping {name} as there are no ground truth labels in this slice")
        continue

    if np.sum(cnn_prediction) == 0:
        print(f"Skipping {name} as there are no CNN predictions in this slice")
        continue

    surface_distances = compute_surface_distances(gt, cnn_prediction, spacing_mm=in_spacing)

    distance_measures[name]['msd'] = compute_average_surface_distance(surface_distances)
    distance_measures[name]["HD95"] = compute_robust_hausdorff(surface_distances, 95)
    distance_measures[name]["DSC"] = compute_dice_coefficient(gt, cnn_prediction)

    print(f"{name}\n\tMSD: {distance_measures[name]['msd']:.3f} mm\n\tHD95: {distance_measures[name]['HD95']:.3f} mm\n\tDSC: {distance_measures[name]['DSC']:.3f}\n")



Let's now run the evaluation over the entire test set!

In [ ]:
MSDs = {name : [] for name in structure_names[1:]}
HD95s = {name : [] for name in structure_names[1:]}
DSCs = {name : [] for name in structure_names[1:]}
failures = {name : 0 for name in structure_names[1:]}
organ_labels = [2,3,4,5]

## Now create some the test dataset and dataloader
spacings_test = np.array(list(test_pixel_sizes.values()))
test_dataset = LCTSCDataGen(window_levelled_slices_test, test_mask_slices, val_transforms, spacings_test)
## Create dataloaders from these datasets
test_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=1)


for in_image, in_mask, in_spacing in test_dataloader:
    # take the first example from the batch
    in_image = in_image[0].unsqueeze(0)
    all_gt = in_mask[0].numpy().astype(float)
    in_spacing = in_spacing[0].numpy()

    test = pl_model.model.predict(in_image).detach() ## the model dosn't do softmax activation, so we have to do it ourselves
    probs = torch.nn.functional.softmax(test, dim=1)[0].numpy().squeeze()
    test_mask = np.argmax(probs, axis=0).astype(float)
    
    for name, organ_label in zip(structure_names[1:], organ_labels):
        cnn_prediction = test_mask == organ_label
        gt = all_gt == organ_label

        if np.sum(gt) == 0:
            continue

        if np.sum(cnn_prediction) == 0:
            failures[name] += 1
            continue

        surface_distances = compute_surface_distances(gt, cnn_prediction, spacing_mm=in_spacing)
        MSDs[name].append(compute_average_surface_distance(surface_distances))
        HD95s[name].append(compute_robust_hausdorff(surface_distances, 95))
        DSCs[name].append(compute_dice_coefficient(gt, cnn_prediction))


print("Mean MSDs")
for name in structure_names[1:]:
    print(f"\t{name}: {np.mean(MSDs[name]):.3f} mm")

print("\nMean HD95s")
for name in structure_names[1:]:
    print(f"\t{name}: {np.mean(HD95s[name]):.3f} mm")

print("\nMean DSCs")
for name in structure_names[1:]:
    print(f"\t{name}: {np.mean(DSCs[name]):.3f}")

print("\nFailures")
for name in structure_names[1:]:
    print(f"\t{name}: {failures[name]}")

# 7. Visualisation

At this point, take some time visualising the results using matplotlib. **We'd like to see some boxplots of the results for each organ.**

In [ ]:
fig, ax = plt.subplots()

### Your code here...

# 8. A new dataset

Let's now look at a new anatomical site ... the head and neck (HnN)

First of all let's try running our previous model on this new dataset.

In [ ]:
## See if we're in colab...
if IN_COLAB:
  hn_datapath = "/content/HnN_data/"
else:
  hn_datapath = "/home/ed/autoseg_workshop_2023/HnN_data/"  ## <---- You will need to change this if running locally

hn_fnames = sorted(getFiles(join(hn_datapath, 'train', 'ims')))
hn_image = window_level(np.load(join(hn_datapath, 'train', 'ims', hn_fnames[23])))
hn_mask = np.load(join(hn_datapath, 'train', 'masks', hn_fnames[23]))
hn_image = val_transforms(image=hn_image[np.newaxis, np.newaxis])['image']
hn_mask[hn_mask == 0.0] = np.nan 
test = pl_model.model.predict(torch.tensor(hn_image)).detach() 
probs = torch.nn.functional.softmax(test, dim=1)[0].numpy().squeeze()
test_mask = np.argmax(probs, axis=0).astype(float)
test_mask[test_mask == 0.0] = np.nan

## Show the results
fig =  plt.figure(figsize=(10,10)) 
ax_gt = fig.add_subplot(121)
ax_gt.set_title("Ground Truth")
ax_cnn = fig.add_subplot(122)
ax_cnn.set_title("CNN contour")

ax_gt.imshow(hn_image.squeeze(), cmap='Greys_r')
ax_gt.imshow(hn_mask.squeeze(), alpha=0.75, cmap='viridis', vmin=0, vmax=5)
ax_gt.invert_yaxis()

ax_cnn.imshow(hn_image.squeeze(), cmap='Greys_r')
ax_cnn.imshow(test_mask, alpha=0.75, cmap='viridis', vmin=0, vmax=5)
ax_cnn.invert_yaxis()


As you can see the abdominal segmentation model does terribly in the head and neck images. This is because the model has never been trained on this anatomical site. We'll have to train a new Head and Neck specific model to handle this new data.

### To do this, go to Part 2!